# 1. 배/전신 제외

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException


import pandas as pd
import numpy as np

from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys

In [ ]:
from selenium import webdriver

chrome_options = webdriver.ChromeOptions()             # 웹 드라이버 옵션 지정
chrome_options.add_argument('--headless')              # 웹 브라우저 띄우지 않음
chrome_options.add_argument('--no-sandbox')            # 별도 웹페이지 작동 해제
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
wd = webdriver.Chrome(options=chrome_options)
time.sleep(random.uniform(1,3))
wd.get(url)

In [ ]:
wait_time = 10  # 대기 시간 설정 (초 단위)

In [ ]:
part_id_names = {
    "B000001": "골반",
    "B000002": "귀",
    "B000003": "기타",
    "B000004": "눈",
    "B000005": "다리",
    "B000006": "등/허리",
    "B000007": "머리",
    "B000008": "목",
    "B000009": "발",
#     "B000010": "배",
    "B000011": "생식기",
    "B000012": "손",
    "B000013": "엉덩이",
    "B000014": "유방",
    "B000015": "입",
#     "B000016": "전신",
    "B000017": "코",
    "B000018": "팔",
    "B000019": "피부",
    "B000020": "가슴",
    "B000021": "얼굴",
}

# 결과를 저장할 리스트 초기화
results = []

start_part_id = "B000001"
end_part_id = "B000022"
current_part_id = start_part_id

# 파트 ID가 종료 ID에 도달할 때까지 반복
while current_part_id != end_part_id:
    url = "https://www.amc.seoul.kr/asan/healthinfo/disease/diseaseList.do?partId={}".format(current_part_id)
    wd.get(url)

    # 페이지 로딩 대기
    page_bar = WebDriverWait(wd, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#content > div.healthinfoWrap > div.pagingWrapSec > span')))
    pages = page_bar.find_elements(By.CSS_SELECTOR, 'a')

    for page_num in range(1, len(pages) + 1):
        xpath = '//*[@id="content"]/div[2]/div[2]/span/a[{}]'.format(page_num)
        wd.find_element(By.XPATH, xpath).click()

        # 질환 데이터 추출
        name_selector = "#listForm > div > div > ul > li > div.contBox > strong"
        names = wd.find_elements(By.CSS_SELECTOR, name_selector)

        for i in range(1, len(names) + 1):
            xpath = '//*[@id="listForm"]/div/div/ul/li[{}]/div[2]/strong/a'.format(i)
            button = wd.find_element(By.XPATH, xpath)
            button.click()

            # 각 질환의 정보 추출
            name_selector = "#content > div.healthinfoWrap.clearfix > div.regionReviewLeft > div.otherRegionBox > ul > li > div.contBox > strong"
            symptom_selector = '#content > div.healthinfoWrap.clearfix > div.regionReviewLeft > div.contDescription > dl > dd:nth-child(6)'
            related_diseases_selector = '#content > div.healthinfoWrap.clearfix > div.regionReviewLeft > div.otherRegionBox > ul > li > div.contBox > dl > dd:nth-child(4)'
            department_selector = '#content > div.healthinfoWrap.clearfix > div.regionReviewLeft > div.otherRegionBox > ul > li > div.contBox > dl > dd:nth-child(6)'
            define_selector = '#content > div.healthinfoWrap.clearfix > div.regionReviewLeft > div.contDescription > dl > dd:nth-child(2) '
            cause_selector = '#content > div.healthinfoWrap.clearfix > div.regionReviewLeft > div.contDescription > dl > dd:nth-child(4) '

            # 요소들 찾기
            names = wd.find_elements(By.CSS_SELECTOR, name_selector)
            symptoms = wd.find_elements(By.CSS_SELECTOR, symptom_selector)
            related_diseases = wd.find_elements(By.CSS_SELECTOR, related_diseases_selector)
            departments = wd.find_elements(By.CSS_SELECTOR, department_selector)
            defines = wd.find_elements(By.CSS_SELECTOR, define_selector)
            causes = wd.find_elements(By.CSS_SELECTOR, cause_selector)

            # 결과를 리스트에 저장
            for name, symptom, related_disease, department, define, cause in zip(names, symptoms, related_diseases, departments, defines, causes):
                results.append({
                    "부위": part_id_names[current_part_id],
                    "질환명": name.text,
                    "정의": define.text if define else None,
                    "원인": cause.text if cause else None,
                    "증상": symptom.text if symptom else None,
                    "관련 질환": related_disease.text if related_disease else None,
                    "진료과": department.text if department else None
                })

            # 이전 페이지로 이동
            wd.back()

    # 다음 파트 ID로 이동
    current_part_id = "B" + str(int(current_part_id[1:]) + 1).zfill(6)

# 데이터프레임으로 변환
df = pd.DataFrame(results)

# 웹 드라이버 종료
wd.quit()

In [ ]:
df = pd.DataFrame(results)

In [ ]:
df.head()

In [ ]:
df.to_csv('./질환증상크롤링_배전신제외.csv', index=False)

# 2. 배/ 전신

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import selenium
from selenium import webdriver
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By # 셀레니움 4.0부터 포함된 함수(필수)
from selenium.webdriver.chrome.service import Service

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

In [ ]:
def create_driver():
    service = Service()
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=service, options=options)
    return driver

In [ ]:
driver = create_driver()
### url 수정 ###
# 배 1페이지 (~10)
# https://www.amc.seoul.kr/asan/healthinfo/disease/diseaseList.do?pageIndex=1&partId=B000010&diseaseKindId=&searchKeyword=%EB%B0%B0
# 배 11페이지
# https://www.amc.seoul.kr/asan/healthinfo/disease/diseaseList.do?pageIndex=11&partId=B000010&diseaseKindId=&searchKeyword=%EB%B0%B0#content
# 전신 1페이지 (~10)
# https://www.amc.seoul.kr/asan/healthinfo/disease/diseaseList.do?pageIndex=1&partId=B000016&diseaseKindId=&searchKeyword=%EC%A0%84%EC%8B%A0
# 전신 11페이지 (~20)
# https://www.amc.seoul.kr/asan/healthinfo/disease/diseaseList.do?pageIndex=11&partId=B000016&diseaseKindId=&searchKeyword=%EC%A0%84%EC%8B%A0

url = '해당 url'
driver.get(url)

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
result = []
for j in range(1, 11): ## (1, page+1) 수정
    ul_element = driver.find_element(By.XPATH, '//*[@id="listForm"]/div/div/ul')
    li_elements = ul_element.find_elements(By.TAG_NAME, 'li')

    for i in range(1, len(li_elements)+1):  
        xpath = f'//*[@id="listForm"]/div/div/ul/li[{i}]/div[2]/strong/a'
        name_btn = driver.find_element(By.XPATH, xpath)
        name_btn.click()

        time.sleep(2)

        #질환명
        name = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[1]/ul/li/div[2]/strong').text
        
        # 정의  
        try:
            definition = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[2]/dl/dd[1]/p[1]').text
        except NoSuchElementException:
            definition = None
            
        #원인
        try:
            cause = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[2]/dl/dd[2]/p').text
        except NoSuchElementException:
            cause = None
            
        # 증상 
        try:
            symptom = driver.find_element(By.XPATH,'//*[@id="content"]/div[2]/div[1]/div[2]/dl/dd[3]/p[1]').text
        except NoSuchElementException:
            symptom = None

        # 관련질환   
        try:
            related = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[1]/ul/li/div[2]/dl/dd[2]').text
        except NoSuchElementException:
            related = None

        # 진료과
        try:
            department = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/div[1]/ul/li/div[2]/dl/dd[3]/a').text
        except NoSuchElementException:
            department = None

        


        result.append(pd.DataFrame({'부위':'전신', ## 부위명 변경
                                       '질환명':name,
                                        '정의':definition,
                                        '원인':cause,
                                     '증상':symptom,
                                       '관련질환':related,
                                      '진료과':department}, index=range(0,1)))


        driver.back()

    btn = f'//*[@id="content"]/div[2]/div[2]/span[2]/a[{j+1}]'
    page_btn = driver.find_element(By.XPATH, btn)
    page_btn.click()
    time.sleep(2)    

In [ ]:
pd.concat(result).reset_index(drop=True)

In [ ]:
import pandas as pd

df = pd.concat(result).reset_index(drop=True)

df.to_csv('질환증상크롤링_배전신.csv', index=False)

## 3. csv 합치기

In [ ]:
df1 = pd.read_csv('./질환증상크롤링_배전신제외.csv')
df2 = pd.read_csv('./질환증상크롤링_배전신.csv')

merged_df = pd.concat([df1, df2], ignore_index=True)

merged_df.to_csv('./질환증상크롤링.csv', index=False)